## Original function definition

(indent modified)

In [1]:
function my_deferred_acceptance(m_prefs, f_prefs)
    #(m_prefs、f_prefs）を渡すとm_matchedとf_matchedが返る関数

    m=length(m_prefs)
    n=length(f_prefs)

    #全ての要素が０の配列を作る
    m_matched=Vector{Int64}(m)
    f_matched=Vector{Int64}(n)
    m_matched[:]=0
    f_matched[:]=0

    #男のインデックスと女のインデックスを渡すと、その男の順位が返ってくる関数（男が選好リストに入っていなければ０が返る）
    function ranking(man ,woman)
        if !(any(man in f_prefs[woman][i] for i in 1:length(f_prefs[woman])))
            return 0
        #女性の選好の中に男性が入っていなければ０を返す
        else
            for i in 1:length(f_prefs[woman])
                if f_prefs[woman][i]==man
                    return i
        #女性からの男性の順位を返す
                end
            end
        end
    end


    for j in 1:n
    #男性の選好リストの１巡目、2巡目…
        for i in 1:m
            #各男性が順に動く
            if n>=m&&all(m_matched.!=0)
            #もし女の方が多く、男が全員マッチしていたら次の人に入らずmatchedを返す
                break
                return m_matched, f_matched
            else
                if j<=length(m_prefs[i])&&m_matched[i]==0
                #ある男性がj巡目に、まだ選好を持ち、相手がいない
                    like=m_prefs[i][j]
                    #iさんj巡目のプロポーズ相手をlikeと定義する

                    if ranking(i, like)!=0
                    #プロポーズ相手の女性の選好リストにiさんが載っている
                        if  f_matched[like]==0
                            m_matched[i]=like
                            f_matched[like]=i
                        #女性に相手がいなければ、#マッチしているリストに追加
                        elseif ranking(i, like)<ranking(f_matched[like],like)
                            m_matched[f_matched[like]]=0
                            m_matched[i]=like
                            f_matched[like]=i
                        #既にマッチしている人より順位が高い（数字が小さい）とき、既にマッチしていた組を変更する
                        end
                    end
                end
            end
        end
    end

    return m_matched, f_matched
end


my_deferred_acceptance (generic function with 1 method)

## Some modification

### 1.

In line 8-11, 

```julia
m_matched=Vector{Int64}(m)
f_matched=Vector{Int64}(n)
m_matched[:]=0
f_matched[:]=0
```

this is equivalent to

```julia
m_matched = zeros(Int64, m)
f_matched = zeros(Int64, n)
```

e.g.

In [6]:
m_matched = zeros(Int64, 3)

3-element Array{Int64,1}:
 0
 0
 0

### 2.

In line 15,

```julia
if !(any(man in f_prefs[woman][i] for i in 1:length(f_prefs[woman])))
```

can be simply written as

```julia
if !(man in f_prefs[woman])
```

In [9]:
f_prefs = [[2, 3], [2, 3, 4, 1], [4, 1, 2]]
man = 3

woman = 1
println( !(man in f_prefs[woman]) )

woman = 3
println( !(man in f_prefs[woman]) )

false
true


### 3.

In line 33-36,

```julia
if n>=m&&all(m_matched.!=0)
    #もし女の方が多く、男が全員マッチしていたら次の人に入らずmatchedを返す
    break
    return m_matched, f_matched
```

* ```n>=m``` is unnecessary. もし n<m だとしても, 男性が全員女性とマッチしていたらその時点で男性側からのproposeはなくなるので, アルゴリズムは終了.

* ```break``` するとその後のreturn... は実行されず, 一番内側のfor loop(for i in 1:m)が終了してしまう. よってbreakを取る

Then

```julia
if all(m_matched.!=0)
    #もし女の方が多く、男が全員マッチしていたら次の人に入らずmatchedを返す
    return m_matched, f_matched
```

### 4. 

In line 38,

```julia
if j<=length(m_prefs[i])&&m_matched[i]==0
```

この条件は少し問題がある. 例えば男性の集合Mと女性の集合Wを 
$M=\{m_1, m_2, m_3\},\ W=\{w_1, w_2, w_3\}$
として, 次のようなpreferenceを考える:

| $\mathbf{m_1}$ | $\mathbf{m_2}$ | $\mathbf{m_3}$ |
|:-----------:|:------------:|:------------:|
| $w_1$ | $w_2$ | $w_2$ |
| $w_2$ | $w_1$ | $w_3$ |

| $\mathbf{w_1}$ | $\mathbf{w_2}$ | $\mathbf{w_3}$ |
|:-----------:|:------------:|:------------:|
| $m_2$ | $m_1$ | $m_3$ |
| $m_1$ | $m_3$ | $m_2$ |

このpreferenceのもとでMen-Proposing DAを行うと

* j=1 のループ終了後のmatching: ($m_1$, $w_1$), ($m_3$, $w_2$)
* j=2 のループ終了後のmatching: ($m_2$, $w_1$), ($m_3$, $w_2$)
* j=3 のループ終了後のmatching: ($m_1$, $w_2$), ($m_2$, $w_1$), ($m_3$, $w_3$)

となる. この例から, (`len(m_prefs[1]) == 1`だが) 実際にはj=3の時も$m_1$のmatchingは変更されうることがわかる. 

男性全員が女性全員に1度ずつproposeすればDAが終了することは保証される(∵ DAのプロセスで1度breakしたmatchingが後で復活することはない)ので, j=1〜nまでループすればアルゴリズムは終了する.

しかしj=Jのとき, 「J番目以下のpreferenceしかmatchingに関係しない」とはいえない.